# ML Analysis

In [1]:
import pandas as pd
import numpy as np

## Data
Data are from 'data_cleaning.ipynb', which uses a subset of merged TRACE and Mergent FISD daily bond data. It follows the sample exclusion criteria of Bai, Bali & Wen (2019) and aggregates returns to monthly based on their methodologies.

In [6]:
data = pd.read_csv('monthly_data.csv', index_col=0)

In [7]:
data.head()

,date,issue_id,cusip_id,volume,issuer_id,prospectus_issuer_name,maturity,offering_amt,interest_frequency,cusip,...,rating_spr,rating_mdy,amount_outstanding,tau,age,ytm,duration,tr_dirty_price,tr_ytm,return
0,2002-08-31,98694,010392DN5,25000.0,86.0,ALABAMA PWR CO,2004-08-15,250000.0,2.0,010392DN5,...,A,A2,250000.0,1.972603,3.038356,3.672086,1.872172,109.77133,2.142072,0.031350
1,2002-08-31,99961,010392DP0,15000.0,86.0,ALABAMA PWR CO,2007-10-01,200000.0,2.0,010392DP0,...,A,A2,200000.0,5.101370,2.931507,4.540522,4.299664,120.46437,3.339660,0.065160
2,2002-08-31,144179,00079FAW2,20000.0,37284.0,ABN AMRO BK N V,2003-12-19,4500.0,2.0,00079FAW2,...,AA,Aa2,4500.0,1.315068,0.200000,12.814093,1.230101,115.34654,1.844255,0.130164
3,2002-08-31,97686,013104AE4,215000.0,93.0,ALBERTSONS INC,2009-08-01,350000.0,2.0,013104AE4,...,BBB+,Baa1,350000.0,6.936986,3.098630,5.601379,5.647303,119.27208,3.834566,0.055352
4,2002-08-31,102864,029050AB7,2000000.0,33708.0,AMERICAN PLUMBING & MECHANICAL,2008-10-15,125000.0,2.0,029050AB7,...,B-,B3,125000.0,6.142466,2.734246,29.812586,3.527641,148.25096,3.569999,-0.324551


Following Moritz and Zimmerman (2016), the goal is to estimate the expected return of stock i in time period t+1, conditional on information in period t.

In the context of corporate bonds, our aim will be to predict the return of bond i. Then based on the predicted return for t+1, bonds are sorted into deciles based on the expected return, and the trading strategy goes long the top decile (ie. highest predicted returns) and short the bottom decile. The out-of-sample test is implemented on a rolling basis: the model is estimated every year, using data from the past 5 years. Returns are predicted for the next 12 months, and in each month the decile portfolio strategy is implemented.

Given the bond characteristics, such as credit rating, duration, and yield-to-maturity (which are all known factors to affect returns), three types of return models can be implemented. The first can focus on raw returns, the second could focus on returns in "excess" of these other factors, and the third can incorporate these characteristics and return-based factors simultaneously. The first model ignores potential correlation between the return-based factors and other bond characteristics, however it follows Moritz and Zimmerman's methodology. The second model would involve first orthogonalizing returns to the bond factors (using linear regression), and then predicting the residual returns; this is appealing, but introduces estimation uncertainty (ie. what if the relationship is non-linear). Lastly, the third methodology is the most flexible. For example, it may be optimal to first sort on duration, and then on past returns etc. 


Thus, the outcome variables (ie. to be predicted) is the next month's return, up to the next 11 months. To simplify things, it may be worthwhile to consider a buy-and-hold strategy: estimate the model, predict the next cummulative X month return (say 3 or 12 months), sort stocks, and go long/sort the portfolios with a holding period of X months. This would have significantly less turnover and hence less transaction costs. This could be used as a robustness check. 

The predictor variables are the past 24 months of 1 month returns (ie. the return over month t-k to t-k+1 for k = (1,...24)). The notation used by Moritz and Zimmerman is $ R_{i,t}(k,1) $. Thus the tree-based model effectively estimates a more complicated version of:

$
r_{i,t+1} = \mu_{1t} I(R_{i,t}(k,1) \lt \tau_t) + \mu_{2t} I(R_{i,t}(k,1) \ge \tau_t)
$

Where $\mu_1t$ can be interpreted as the return on a portfolio for which all stocks $i$ satisfy $R_{i,t}(k,1) \lt \tau_t$ in period $t$. Hence instead of simply sorting on a particular decile, stocks are sorted deeply on potentially many different criteria.

In terms of implementation practicicality, the ML model can be estimated akin to the Fama-MacBeth procedure. Using the past 5 years (60 months) of data, predictions can be formed. The estimation is over the time-dimension (in an OLS framework equivalent to averaging the results of 60 cross-sectional regressions), while the prediction is over the cross-sectional dimension (ie. bonds are ranked by highest return in the next cross-section.)